In [1]:
!pip install bs4

In [24]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# URL de la lista de pasajeros
url = "https://titanicfacts.net/titanic-passenger-list/"

# Realizar la solicitud HTTP
response = requests.get(url)

# Comprobar si la solicitud fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extraer los nombres de los pasajeros
    passenger_list = soup.find_all('tr')
    
    # Lista para almacenar los datos
    data = []

    for passenger in passenger_list:
        columns = passenger.find_all('td')
        if columns:
            surname = columns[0].text.strip()
            first_names = columns[1].text.strip()
            age = columns[2].text.strip()
            boarded = columns[3].text.strip()
            status = columns[4].text.strip()

            # Convertir el valor de "status" a "1" o "0"
            status_numeric = "1" if status == "S" else "0" if status == "†" else None

            # Añadir los datos a la lista
            data.append([surname, first_names, age, boarded, status_numeric])

    # Crear un DataFrame con los datos
    df = pd.DataFrame(data, columns=["Surname", "FirstName", "Age", "Boarded", "Status"])
    
    # Mostrar el DataFrame
    print(df)
    
else:
    print(f"Error al acceder a la página: {response.status_code}")


                 Surname                   FirstName  Age      Boarded Status
0                Surname                 First Names  Age      Boarded   None
1                  Allen       Miss Elisabeth Walton   29  Southampton      1
2                Allison  Mr Hudson Joshua Creighton   30  Southampton      0
3                Allison            Mrs Bessie Waldo   25  Southampton      0
4                Allison          Miss Helen Loraine    2  Southampton      0
...                  ...                         ...  ...          ...    ...
1315  Youssiff (Sam’Aan)                   Mr Gerios   45    Cherbourg      0
1316            Zakarian                    Mr Ortin   27    Cherbourg      0
1317            Zakarian              Mr Mapriededer   22    Cherbourg      0
1318               Zenni                   Mr Philip   22    Cherbourg      1
1319          Zimmermann                      Mr Leo   29  Southampton      0

[1320 rows x 5 columns]


In [36]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
combine = [train_df, test_df]

In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [26]:
import re

# Función para eliminar texto entre paréntesis
def clean_firstname(name):
    # Eliminamos cualquier texto entre paréntesis
    return re.sub(r'\(.*?\)', '', name).strip()

# Separar Surname y FirstName
def separate_name(df):
    # Dividimos 'Name' en Surname y el resto del nombre (incluyendo títulos)
    df[['Surname', 'FullName']] = df['Name'].str.split(',', n=1, expand=True)  # Aquí usamos n=1 para limitar la división
    
    # Limpiar el campo 'FullName' eliminando títulos y nombres entre paréntesis
    df['FirstName'] = df['FullName'].apply(clean_firstname).str.strip()
    
    # Eliminamos la columna temporal 'FullName'
    df.drop(columns=['FullName'], inplace=True)
    
    return df

# Aplicamos la función de separación de nombres
separado_df = separate_name(train_df)

In [31]:
# Función para llenar las edades faltantes
def fill_missing_age(test_df, df):
    # Recorremos las filas del test_df
    for index, row in test_df.iterrows():
        if pd.isnull(row['Age']):  # Si el valor de 'Age' es nulo
            # Separamos el campo 'Name' en Surname y FirstName
            surname, first_name = row['Name'].split(',')
            
            # Buscamos en df por el apellido y nombre
            match = df[(df['Surname'] == surname) ]
            
            # Si encontramos una coincidencia en df, asignamos la edad
            if not match.empty:
                test_df.at[index, 'Age'] = match.iloc[0]['Age']
    
    return test_df

# Llenamos los valores faltantes
test_df_filled = fill_missing_age(separado_df, df)

# Mostrar el resultado
# tools.display_dataframe_to_user(name="Updated Test DataFrame", dataframe=test_df_filled)

/tmp/ipykernel_2456/2126827017.py:14: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '28' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  test_df.at[index, 'Age'] = match.iloc[0]['Age']


In [32]:
# Supongamos que 'df' es tu DataFrame
null_counts = test_df_filled.isnull().sum()

# Mostrar la cantidad de valores nulos por columna
print(null_counts)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             40
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Surname          0
FirstName        0
dtype: int64
